# **Comparing the best Portfolio of Q4 2024 on Q1 2025**

Let's see if the portfolio with the best sharpe ratio of our simulations in Q4 2024 is still the best in Q1 2025.

All data was generated with the `fetch_data.py` script, which fetches the data from Yahoo Finance and saves it in a CSV file. The data is then loaded into a pandas DataFrame and used to create the portfolio.

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

# **Loading the best Sharpe Ratio Portfolio**

In [2]:
# Read all files that start with "output" in the current directory and create a dataframe
columns = [f'T{i}' for i in range (1, 26)] + [f'W{i}' for i in range (1, 26)] + ['Sharpe']

df = pd.DataFrame(columns=columns)

for file in os.listdir('.'):
    if file.startswith('output'):
        # Read the file and append it to the dataframe
        temp_df = pd.read_csv(file, sep=',')
        temp_df.columns = columns
        df = pd.concat([df, temp_df], ignore_index=True)

/tmp/ipykernel_472712/3429224788.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, temp_df], ignore_index=True)


In [10]:
# Find the best Sharpe ratio and get the row
best_sharpe_row = df.loc[df['Sharpe'].idxmax()]

# Extract the tickers
tickers = best_sharpe_row[:25].values

# Extract the weights
weights = best_sharpe_row[25:50].values

In [11]:
tickers, weights

(array(['WMT', 'V', 'UNH', 'TRV', 'PG', 'NVDA', 'NKE', 'MSFT', 'MRK',
        'MCD', 'JPM', 'JNJ', 'IBM', 'HON', 'HD', 'GS', 'DIS', 'CVX',
        'CSCO', 'CRM', 'CAT', 'BA', 'AXP', 'AMZN', 'AAPL'], dtype=object),
 array([np.float64(0.0974281874714393), np.float64(0.0912326465137112),
        np.float64(0.0159919688972767), np.float64(0.0206034243612881),
        np.float64(0.0466752713413208), np.float64(0.081652134082355),
        np.float64(1.938109111300308e-05), np.float64(0.0126888024607757),
        np.float64(0.0028061776849342), np.float64(0.0010453306182586),
        np.float64(0.0118831308637931), np.float64(0.0035618777464765),
        np.float64(0.0940934381132161), np.float64(0.0435450434955281),
        np.float64(0.0475790623004446), np.float64(0.0491659353953422),
        np.float64(0.1011535188592612), np.float64(0.0187989575130129),
        np.float64(0.0246768774556158), np.float64(0.0940807807937921),
        np.float64(0.0324204502066476), np.float64(0.01579035493

# **Apply the best portfolio weights to the Q1 2025 data**

In [17]:
closing_prices = pd.read_csv('dow_jones_close_prices_jan_mar_2025.csv', sep=',')

closing_prices.head()

,Date,AAPL,AMGN,AMZN,AXP,BA,CAT,CRM,CSCO,CVX,...,MSFT,NKE,NVDA,PG,SHW,TRV,UNH,V,VZ,WMT
0,2025-01-02,243.582199,257.202057,220.220001,296.744843,171.869995,356.738220,330.141266,58.310692,145.100601,...,417.742371,73.299004,138.297272,163.952057,332.433472,239.908493,502.363068,313.871857,38.885319,89.537041
1,2025-01-03,243.092728,259.126526,224.190002,302.077148,169.899994,360.724365,332.377747,58.469631,146.228088,...,422.502838,72.940819,144.456711,163.112457,335.806122,241.302780,510.816956,314.381012,38.933670,90.313026
2,2025-01-06,244.730927,256.517578,227.610001,300.881134,170.779999,361.130890,330.011475,58.380230,145.644547,...,426.993835,71.637421,149.416245,158.637802,337.083313,240.894455,511.484070,312.514160,38.305088,90.959686
3,2025-01-07,241.944000,259.959778,222.110001,301.020660,172.509995,359.941010,324.420258,58.539169,147.948990,...,421.524780,71.726959,140.127106,159.329254,333.670776,240.695267,512.061646,311.146454,37.637817,90.342873
4,2025-01-08,242.433441,262.092529,222.130005,301.847931,171.759995,358.027283,326.387146,58.807377,148.651215,...,423.710419,70.930992,140.097122,160.119476,336.803925,241.780838,522.287964,312.074890,37.657158,91.327789


In [15]:
# Filter the closing prices columns to only include the tickers
closing_prices_filtered = closing_prices[['Date'] + list(tickers)]

closing_prices_filtered.shape

(59, 26)

In [22]:
# Transform into a matrix of the closing prices
closing_prices_matrix = closing_prices_filtered.iloc[:, 1:].values

# Calculate the daily returns
daily_returns = np.zeros(closing_prices_matrix.shape)

for i in range(1, closing_prices_matrix.shape[0]):
    daily_returns[i] = (closing_prices_matrix[i] - closing_prices_matrix[i-1]) / closing_prices_matrix[i-1]

daily_returns = daily_returns[1:]

# Calculate the weighted daily returns
weighted_daily_returns = daily_returns.dot(weights)

ann_return = weighted_daily_returns.mean() * 252

ann_return

np.float64(-0.1823327990537905)

In [34]:
cov = np.cov(daily_returns, rowvar=False)

daily_vol = np.sqrt(weights.T @ cov @ weights)

ann_vol = daily_vol * np.sqrt(252)

ann_vol

np.float64(0.16655735940887106)

In [35]:
sharpe = ann_return / ann_vol

print(f"Annualized Return: {ann_return:.4f}")
print(f"Annualized Volatility: {ann_vol:.4f}")
print(f"Sharpe Ratio: {sharpe:.4f}")

Annualized Return: -0.1823
Annualized Volatility: 0.1666
Sharpe Ratio: -1.0947
